In [13]:
!pip install kaggle

In [14]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [0]:
import json
token = {"username":"kaggle_username","key":"kaggle_api_key"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!chmod 600 /content/.kaggle/kaggle.json
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [17]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [18]:
!kaggle competitions download -c quora-insincere-questions-classification -p /content

100% 5.95G/5.96G [00:52<00:00, 134MB/s]
100% 5.96G/5.96G [00:52<00:00, 121MB/s]
  0% 0.00/649k [00:00<?, ?B/s]
100% 649k/649k [00:00<00:00, 87.1MB/s]
  0% 0.00/2.36M [00:00<?, ?B/s]
100% 2.36M/2.36M [00:00<00:00, 158MB/s]
 85% 46.0M/54.4M [00:00<00:00, 50.9MB/s]
100% 54.4M/54.4M [00:00<00:00, 84.2MB/s]


In [19]:
# Install PyTorch

from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
  
# Install pytorch-pretrained-BERT

!pip install pytorch-pretrained-bert

# Install torchsample

!pip install git+https://github.com/ncullen93/torchsample
  
# Install gplearn

!pip install gplearn

  Cloning https://github.com/ncullen93/torchsample to /tmp/pip-req-build-uwo801p8
  Running setup.py bdist_wheel for torchsample ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-4ogw3gtp/wheels/88/c7/72/14cd9a173eed1e29d0b17d866e7d9ee511d31a834aedd27489
Successfully built torchsample


In [20]:
!unzip \*.zip

Archive:  test.csv.zip
  inflating: test.csv                

Archive:  train.csv.zip
  inflating: train.csv               

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   

Archive:  embeddings.zip
   creating: GoogleNews-vectors-negative300/
   creating: glove.840B.300d/
   creating: paragram_300_sl999/
   creating: wiki-news-300d-1M/
  inflating: glove.840B.300d/glove.840B.300d.txt  
  inflating: GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin  
  inflating: wiki-news-300d-1M/wiki-news-300d-1M.vec  
  inflating: paragram_300_sl999/README.txt  
  inflating: paragram_300_sl999/paragram_300_sl999.txt  

4 archives were successfully processed.


In [21]:
import numpy as np
import pandas as pd 

import keras
from keras.utils import to_categorical
from keras.preprocessing import sequence

import torch
from torch import LongTensor, FloatTensor, no_grad
from torch.backends import cudnn
from torch.nn import Module, Sequential, Linear, Sigmoid, ReLU, Dropout
from torch.backends import cudnn
from torch.utils import checkpoint
from torch.optim import Adam

import pytorch_pretrained_bert
from pytorch_pretrained_bert import BertForSequenceClassification, BertTokenizer

import gplearn
from gplearn.genetic import SymbolicTransformer

import torchsample
from torchsample.modules import ModuleTrainer

import os
import gc

Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [0]:
train_data_df = pd.read_csv('train.csv')
train_data = train_data_df.values

In [0]:
sentences = []
targets = []

for i in range(0, len(train_data)):
    sentences.append(train_data[i][1])
    targets.append(train_data[i][2])
    
targets = np.array(targets)

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

bag_of_words = []
sequences = []

for i in range(len(sentences)):
    tokenized_sentence = tokenizer.tokenize(sentences[i])
    bag_of_words.append(tokenized_sentence)
    sequences.append(tokenizer.convert_tokens_to_ids(tokenized_sentence))

100%|██████████| 231508/231508 [00:00<00:00, 1976829.98B/s]


In [0]:
MAX_SEQUENCE_LENGTH = 128
TRAIN_VAL_SPLIT = 0.8
NUM_TEXT_FEATURES = 768

In [0]:
sequences = LongTensor(sequence.pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH))
# toxicities = LongTensor(toxicities)
targets = FloatTensor(targets.reshape(len(targets), 1))

In [0]:
train_sequences = sequences[:np.int32(TRAIN_VAL_SPLIT*len(sequences))]
# train_toxicities = toxicities[:np.int32(TRAIN_VAL_SPLIT*len(sequences))]
train_targets = targets[:np.int32(TRAIN_VAL_SPLIT*len(targets))]

val_sequences = sequences[np.int32(TRAIN_VAL_SPLIT*len(sequences)):]
# val_toxicities = toxicities[np.int32(TRAIN_VAL_SPLIT*len(sequences)):]
val_targets = targets[np.int32(TRAIN_VAL_SPLIT*len(targets)):]

In [36]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased').cuda()

100%|██████████| 407873900/407873900 [00:09<00:00, 43786392.24B/s]


In [37]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertInterme

In [0]:
NUM_TRANSFORMER_BLOCKS = 12

In [0]:
# Fine - Tune only 2 Transformer Blocks and the Pooling Layer.
# Fine - Tune all parts of the model (excluding the Embeddings layer) when you have enough GPU !

In [0]:
for param in model.parameters():
  param.requires_grad = False
  
for param in model.bert.encoder.layer[0].parameters():
  param.requires_grad = True
  
for param in model.bert.pooler.parameters():
  param.requires_grad = True
  
for param in model.bert.encoder.layer[NUM_TRANSFORMER_BLOCKS - 1].parameters():
  param.requires_grad = True

In [0]:
model.classifier = Sequential(Linear(in_features=NUM_TEXT_FEATURES, out_features=100), ReLU(), Linear(in_features=100, out_features=20, bias=True), ReLU(), Linear(in_features=20, out_features=1, bias=True), Sigmoid()).cuda()

In [0]:
cudnn.benchmark = True

In [0]:
trainer = ModuleTrainer(model)

In [0]:
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
NUM_EPOCHS = 2

In [0]:
# GOOD BATCH SIZE + LEARNING RATE COMINATIONS :
# 1) 32 AND 0.00001

In [0]:
# Try decreasing the MAX SEQ LENGTH to 64 and increasing the BATCH SIZE to 64

In [0]:
trainer.compile(loss='binary_cross_entropy',
                optimizer=Adam(params=model.parameters(), lr=LEARNING_RATE))

trainer.fit(train_sequences.cuda(), train_targets.cuda(),
            val_data=(val_sequences.cuda(), val_targets.cuda()),
            num_epoch=NUM_EPOCHS,
            batch_size=BATCH_SIZE,
            verbose=1)

Epoch 1/2:   0%|          | 0/32654 [00:00<?, ? batches/s]/usr/local/lib/python3.6/dist-packages/torchsample/modules/module_trainer.py:280: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  batch_logs['loss'] = loss.data[0]
Epoch 1/2:   0%|          | 135/32654 [02:31<10:14:54,  1.13s/ batches, loss=0.2906]